<a href="https://colab.research.google.com/github/ajou-math/growth-prediction/blob/feature%2FmachineLearning/female.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/Shareddrives/growthPrediction/machineLearning/boneage-training-dataset.zip" -d dataset
!ls

dataset  drive	sample_data


In [ ]:
import tensorflow as tf

# female_model = tf.keras.applications.Xception(weights=None, include_top=True, input_shape=(224,224,1), classes=1, classifier_activation=None)
female_model = tf.keras.applications.InceptionV3(weights=None, include_top=True, input_shape=(224,224,1), classes=1, classifier_activation=None)

# print(model.summary())

In [ ]:
female_model.compile(loss='mae', optimizer='adam')

import pandas as pd

csv_train=pd.read_csv('/content/drive/Shareddrives/growthPrediction/machineLearning/boneage-training-dataset.csv')
csv_train.head()

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False


In [ ]:
for i in range(len(csv_train['id'])):
  csv_train.loc[i,"id"] = f"{csv_train['id'][i]}.png"

csv_train.head()


,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [ ]:
female_df = csv_train.loc [csv_train.male != True]
female_df.head()

len(female_df)

5778

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # random으로 validation 뽑음

train_generator = datagen.flow_from_dataframe(female_df, directory='/content/dataset/boneage-training-dataset/boneage-training-dataset', x_col='id', y_col='boneage',
                                              target_size=(224,224), color_mode='grayscale',
                                              class_mode='raw', subset='training')
valid_generator = datagen.flow_from_dataframe(female_df, directory='/content/dataset/boneage-training-dataset/boneage-training-dataset', x_col='id', y_col='boneage',
                                              target_size=(224,224), color_mode='grayscale', 
                                              class_mode='raw', subset='validation', shuffle=False)


Found 4623 validated image filenames.
Found 1155 validated image filenames.


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only=True,
                                                filepath="/content/drive/Shareddrives/growthPrediction/machineLearning/check_point_f")

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = female_model.fit_generator(train_generator, validation_data=valid_generator, epochs=100, callbacks=[checkpoint, early_stopping])

In [ ]:
# 학습 과정에서의 loss와 val_loss 시각화
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# 학습 과정에서의 loss와 val_loss 시각화
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylim(0,100)
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
# 예측값과 정답값을 비교
load_model = tf.keras.models.load_model("/content/drive/Shareddrives/growthPrediction/machineLearning/check_point")

import numpy as np

def changeYear(ARR) :
  NewARR = []
  for num in ARR :  
    year = num / 12
    remainder = year % 1
    
    if remainder <= 0.25 :
      remainder = 0
      NewARR.append(int(year) + remainder)
    elif remainder <= 0.75 :
      remainder = 0.5
      NewARR.append(int(year) + remainder)
    else :
      remainder = 1
      NewARR.append(int(year) + remainder)
  return NewARR

predict = load_model.predict_generator(valid_generator).flatten()
predict_year = changeYear(predict)
answer_year = changeYear(valid_generator.labels)

mae_month = [abs(predict[i] - valid_generator.labels[i]) for i in range(len(predict))]
mae_year = [abs(predict_year[i] - answer_year[i]) for i in range(len(predict_year))]

compare = pd.DataFrame({"id" : valid_generator.filenames,
                        "예측(개월)" : predict,
                        "정답(개월)" : valid_generator.labels,
                        "예측(년)" : predict_year,
                        "정답(년)" : answer_year,
                        "mae(년)" : mae_year})

total_mae_month = np.mean(mae_month)
total_mae_year = np.mean(mae_year)

compare.head(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


,id,예측(개월),정답(개월),예측(년),정답(년),mae(년)
0,1377.png,208.600555,180,17.5,15.0,2.5
1,1378.png,85.598381,12,7.0,1.0,6.0
2,1379.png,114.468216,94,9.5,8.0,1.5
3,1381.png,83.828911,82,7.0,7.0,0.0
4,1388.png,137.212677,126,11.5,10.5,1.0
...,...,...,...,...,...,...
95,1595.png,193.074097,165,16.0,13.5,2.5
96,1599.png,67.268738,42,5.5,3.5,2.0
97,1601.png,61.763264,69,5.0,5.5,0.5
98,1602.png,197.766571,162,16.5,13.5,3.0


In [ ]:
print(total_mae_month, total_mae_year)

32.007598127224746 2.657142857142857


In [ ]:
# 최종 결과값과 예측값 비교 시각화
import matplotlib.pyplot as plt

plt.scatter(valid_generator.labels, predict, color='b', s=3)
plt.plot(valid_generator.labels, valid_generator.labels, 'r')
plt.xlabel("Answer")
plt.ylabel("Prediction")
plt.show()